In [1]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),  
    transforms.ToTensor(),           
])

In [2]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder("data/train", transform=transform)
test_dataset  = ImageFolder("data/test",  transform=transform)


In [3]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(train_dataset.class_to_idx)
images, labels = next(iter(train_loader))
print(images.shape, labels.shape)


{'chihuahua': 0, 'muffin': 1}
torch.Size([32, 3, 128, 128]) torch.Size([32])


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MuffinChihuahuaCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [6]:
model = MuffinChihuahuaCNN()
print(model)


MuffinChihuahuaCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=32768, out_features=256, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)


In [7]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
import torch

num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}")


Epoch 1/15, Loss: 33.7361
Epoch 2/15, Loss: 30.2330
Epoch 3/15, Loss: 30.9481
Epoch 4/15, Loss: 26.6885
Epoch 5/15, Loss: 27.9477
Epoch 6/15, Loss: 24.9160
Epoch 7/15, Loss: 23.8337
Epoch 8/15, Loss: 21.3039
Epoch 9/15, Loss: 21.5784
Epoch 10/15, Loss: 20.6280
Epoch 11/15, Loss: 17.1490
Epoch 12/15, Loss: 13.2897
Epoch 13/15, Loss: 21.5995
Epoch 14/15, Loss: 17.9750
Epoch 15/15, Loss: 14.1572


In [13]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy:", correct / total)


Test Accuracy: 0.924831081081081


In [14]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


In [15]:
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [16]:
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)


Confusion Matrix:
 [[574  66]
 [ 23 521]]


In [17]:
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["chihuahua", "muffin"]))



Classification Report:
              precision    recall  f1-score   support

   chihuahua       0.96      0.90      0.93       640
      muffin       0.89      0.96      0.92       544

    accuracy                           0.92      1184
   macro avg       0.92      0.93      0.92      1184
weighted avg       0.93      0.92      0.92      1184

